In [30]:
# Dependencies
import pandas as pd

In [31]:
# Load CSV as Dataframe
wine_data = "Data/winemag-data-130k-v2.csv"
wine_df = pd.read_csv(wine_data, encoding = 'latin-1')

In [71]:
# List off words to look for
fruit_list = ['cherry','plum','apple','blackberry','lemon',
    'raspberry','peach','pear','currant','lime',
    'orange','strawberry','melon','grapefruit','pineapple']
# Build dictionary with words as keys and empty sub-dictionaries as values
country_fruits = {}

# For each wine, check description for fruit words
for entry in wine_df.index:
    country = wine_df['country'][entry]
    if not pd.isna(country):
        for fruit in fruit_list:
            if fruit in wine_df['description'][entry]:
                # Add country if needed
                if country not in country_fruits.keys():
                    country_fruits[country] = dict.fromkeys(fruit_list,0)
                # For each instance of a fruit word, increment the revelant counter for that wine's country
                country_fruits[country][fruit] += 1
    
# Flip resulting dataframe to make narrower
country_fruits_df = pd.DataFrame(country_fruits).transpose()

,cherry,plum,apple,blackberry,lemon,raspberry,peach,pear,currant,lime,orange,strawberry,melon,grapefruit,pineapple
Italy,7973,2341,2371,2036,1045,2087,2288,1738,748,348,575,591,563,474,335
US,13213,4983,7116,5772,4619,4436,3479,3837,4562,3076,2930,2291,1522,1465,2392
Spain,1029,2106,954,935,400,713,459,163,227,362,318,111,559,211,96
Germany,209,192,546,17,625,32,764,186,5,645,138,9,162,308,155
France,1375,1980,2898,767,1516,472,905,1564,2068,459,740,659,422,599,356
Argentina,514,1140,309,794,171,332,159,94,134,110,129,34,288,55,63
Chile,659,1388,522,610,228,392,163,149,190,291,136,37,336,204,88
Australia,292,366,295,113,127,97,160,137,46,130,42,31,147,51,167
Austria,337,363,569,56,717,23,160,853,129,141,144,51,32,254,75
South Africa,283,354,319,120,99,121,169,129,95,129,174,61,186,24,65
